In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0, "../src/")
sys.path.insert(0, "../../../src/")
sys.path.insert(0, "../../../projects/skimpy_bench/src/")
sys.path.insert(0, "../../../projects/skimpy_blox/")

In [ ]:
import skimpy
import skimpy.functional as F
import skimpy_3d

In [ ]:
def blur(tensor):
    G_x = (1 / 64.0) * skimpy.Tensor.from_list([[[1, 6, 15, 20, 15, 6, 1]]]).to(float)
    G_y = (1 / 64.0) * skimpy.Tensor.from_list([[[1], [6], [15], [20], [15], [6], [1]]]).to(float)
    G_z = (1 / 64.0) * skimpy.Tensor.from_list([[[1]], [[6]], [[15]], [[20]], [[15]], [[6]], [[1]]]).to(float)
    tensor = F.conv_3d(tensor, G_x, padding=(3, 0, 0), fill=0.0)
    tensor = F.conv_3d(tensor, G_y, padding=(0, 3, 0), fill=0.0)  
    tensor = F.conv_3d(tensor, G_z, padding=(0, 0, 3), fill=0.0)
    return tensor

In [ ]:
t = skimpy.Tensor(shape=(256, 256, 256), val=0, dtype=float)

In [ ]:
t[64:192, 64:192, 64:192] = 1

In [ ]:
%%time
for i in range(5):
    t = blur(t)

In [ ]:
config = skimpy_3d.VoxelConfig()
config[0] = skimpy_3d.EmptyVoxel()
config[1] = skimpy_3d.ColorVoxel(0, 200, 200)

In [ ]:
%%time
mesh = skimpy_3d.generate_mesh(config, (t > 0.5).to(int)._tensor)

In [ ]:
from pythreejs import *
import numpy as np
from IPython.display import display

In [ ]:
js_mesh = Mesh(
    geometry=BufferGeometry(
        attributes={
            "position": BufferAttribute(
                np.array(mesh.positions, dtype='float32').reshape(-1, 3),
                normalized=False,
            ),
            "normal": BufferAttribute(
                np.array(mesh.normals, dtype='float32').reshape(-1, 3),
                normalized=False,
            ),
            "color": BufferAttribute(
                np.array(mesh.colors, dtype='uint8').reshape(-1, 4)
            ),
            "index": BufferAttribute(
                np.array(mesh.triangles, dtype='uint32'),
                normalized=False,
            ),
        },
    ),
    material=MeshLambertMaterial(vertexColors="VertexColors"),
    position=[0, 0, 0],
)

In [ ]:
js_mesh

In [ ]:
%%time
for i in range(4):
    t = sum([
        0.125 * t[0::2, 0::2, 0::2],
        0.125 * t[1::2, 0::2, 0::2],
        0.125 * t[0::2, 1::2, 0::2],
        0.125 * t[1::2, 1::2, 0::2],
        0.125 * t[0::2, 0::2, 1::2],
        0.125 * t[1::2, 0::2, 1::2],
        0.125 * t[0::2, 1::2, 1::2],
        0.125 * t[1::2, 1::2, 1::2],
    ])

In [ ]:
avg_kernel = skimpy.Tensor.from_list([[[1, 1], [1, 1]], [[1, 1], [1, 1]]]).to(float) / 8
density_lattice = F.conv_3d(
    t,
    avg_kernel,
    padding=1,
)

In [ ]:
%%time
mesh = skimpy_3d.marching_cubes(density_lattice._tensor, 0.5)

In [ ]:
geometry = BufferGeometry(
        attributes={
            "position": BufferAttribute(
                np.array(mesh.positions, dtype='float32').reshape(-1, 3),
                normalized=False,
            ),
            "index": BufferAttribute(
                np.array(mesh.triangles, dtype='uint32'),
                normalized=False,
            ),
        },
    )

geometry.exec_three_obj_method("computeVertexNormals")

In [ ]:
js_mesh = Mesh(
    geometry=geometry,
    material=MeshLambertMaterial(),
    position=[0, 0, 0],
)

In [ ]:
js_mesh

In [ ]:
wf_geometry = WireframeGeometry(geometry)

In [ ]:
js_lines = LineSegments(
    geometry=wf_geometry,
    material=LineBasicMaterial(color="green"),
    position=[0, 0, 0],
)

In [ ]:
js_lines

In [ ]:
camera = PerspectiveCamera(position=(8, 8, 8), lookAt=(0,0,0), fov=20)
scene = Scene(children=[js_lines, camera])

In [ ]:
renderer = Renderer(
    camera=camera, 
    background="white", 
    background_opacity=1,
    scene=scene,
    controls=[OrbitControls(controlling=camera)],
    width=984,
    height=984,
)

display(renderer)